In [296]:
import pandas as pd
import os

In [297]:
path = os.path.join('OriginalData', 'education.csv')
data = pd.read_csv(path)
data_backup = data.copy()

In [299]:
# Show NaN values
data.isnull().sum()

user_id               0
school_name           0
degree              166
fields_of_study      72
start_year_month    922
end_year_month      943
dtype: int64

In [300]:
# Too many NaNs, drop start_year_month and end_year_month
data.drop(['start_year_month', 'end_year_month'], axis=1, inplace=True)
data

,user_id,school_name,degree,fields_of_study
0,0,Anadolu Üniversitesi,NaN,NaN
1,0,Adıyaman Üniversitesi,NaN,NaN
2,0,Fırat Üniversitesi,NaN,NaN
3,1,Fırat Üniversitesi,Yüksek Lisans,Yazılım Mühendisliği
4,1,Fırat Üniversitesi,Lisans,Yazılım Mühendisliği
...,...,...,...,...
995,587,Uniwersytet Łódzki,Bachelor's degree,Bilgisayar Mühendisliği
996,588,Fırat Üniversitesi,NaN,NaN
997,588,Universitatea „Petru Maior” din Târgu-Mureș,Lisans Derecesi,Bilgisayar Bilimleri
998,589,Fırat Üniversitesi,Lisans Derecesi,Adli Bilişim Mühendisliği


In [301]:
# Lowercase all rows in the dataframe
data = data.applymap(lambda s:s.lower() if type(s) == str else s)
data

,user_id,school_name,degree,fields_of_study
0,0,anadolu üniversitesi,NaN,NaN
1,0,adıyaman üniversitesi,NaN,NaN
2,0,fırat üniversitesi,NaN,NaN
3,1,fırat üniversitesi,yüksek lisans,yazılım mühendisliği
4,1,fırat üniversitesi,lisans,yazılım mühendisliği
...,...,...,...,...
995,587,uniwersytet łódzki,bachelor's degree,bilgisayar mühendisliği
996,588,fırat üniversitesi,NaN,NaN
997,588,universitatea „petru maior” din târgu-mureș,lisans derecesi,bilgisayar bilimleri
998,589,fırat üniversitesi,lisans derecesi,adli bilişim mühendisliği


In [302]:
# add a column for each user, with the value of the amount of rows
# that the user has in the dataset
data['education_count'] = data.groupby('user_id')['user_id'].transform('count')

In [303]:
# remove trailing and leading spaces
data['fields_of_study'] = data['fields_of_study'].str.strip()

In [304]:
# Has went to university
data['attended_university'] = data['school_name'].apply(lambda x: 1 if 'üniversite' in x or 'university' in x or 'uni' in x or 'üni' in x or 'univ' in x else 0)
data

,user_id,school_name,degree,fields_of_study,education_count,attended_university
0,0,anadolu üniversitesi,NaN,NaN,3,1
1,0,adıyaman üniversitesi,NaN,NaN,3,1
2,0,fırat üniversitesi,NaN,NaN,3,1
3,1,fırat üniversitesi,yüksek lisans,yazılım mühendisliği,2,1
4,1,fırat üniversitesi,lisans,yazılım mühendisliği,2,1
...,...,...,...,...,...,...
995,587,uniwersytet łódzki,bachelor's degree,bilgisayar mühendisliği,2,1
996,588,fırat üniversitesi,NaN,NaN,2,1
997,588,universitatea „petru maior” din târgu-mureș,lisans derecesi,bilgisayar bilimleri,2,1
998,589,fırat üniversitesi,lisans derecesi,adli bilişim mühendisliği,1,1


In [305]:
from fuzzywuzzy import fuzz
from collections import defaultdict

def group_strings(strings, reference_strings, similarity_threshold):
    groups = defaultdict(list)
    for i, string1 in enumerate(reference_strings):
        for j, string2 in enumerate(strings):
            similarity = fuzz.token_set_ratio(string1, string2)
            if similarity >= similarity_threshold:
                groups[string1].append(string2)
    return groups

In [306]:
# Engineering, IT, science
engineering_groups = ["Mühendis", "Mühendislik", "Müh.", "Mühendisliği", "Engineering", "Engi", "Bilgisayar", "Bilişim", "Bilim", "Teknoloji", "Meka", "Makine", "Machine", "Veri", "inşaat", "science", "sciences", "Web"]

# Business, Marketing etc.
business_group = ["Finance", "Marketing", "Business", "İş", "Pazarlama", "İşletme", ]

# Lower all the groups
engineering_groups = [x.lower() for x in engineering_groups]
business_group = [x.lower() for x in business_group]


skill_groups = engineering_groups + business_group
skill_groups

['mühendis',
 'mühendislik',
 'müh.',
 'mühendisliği',
 'engineering',
 'engi',
 'bilgisayar',
 'bilişim',
 'bilim',
 'teknoloji',
 'meka',
 'makine',
 'machine',
 'veri',
 'inşaat',
 'science',
 'sciences',
 'web',
 'finance',
 'marketing',
 'business',
 'i̇ş',
 'pazarlama',
 'i̇şletme']

In [307]:
skills = data['fields_of_study'].unique().tolist()

In [308]:
skill_groups = group_strings(skills, skill_groups, 50)

In [309]:
# Combine it skills
engineering_fields = []
for key, value in skill_groups.items():
    if key in engineering_groups:
        engineering_fields += value
it_skills = list(set(engineering_fields))

# Combine business skills
business_fields = []
for key, value in skill_groups.items():
    if key in business_group:
        business_fields += value
business_skills = list(set(business_fields))


In [310]:
# Check for any similarities
for a in it_skills:
    if a in business_skills:
        print(a)

science
i̇nşaat mühendisliği
makine mühendisliği i̇le i̇lgili teknolojiler
life sciences
bilgisayar programcılığı(i̇ngilizce)
nursing
web programlama


In [311]:
# Remove NaN values
it_skills = [x for x in it_skills if str(x) != 'nan']
business_skills = [x for x in business_skills if str(x) != 'nan']


In [312]:
for a in business_skills:
    if a in it_skills:
        # Remove from business skills
        business_skills.remove(a)

In [313]:
# View
business_skills

['a1-a2 ,i̇ngilizce eğitimi',
 'i̇nsan kaynakları yönetimi/personel yönetimi, genel',
 'matematik',
 'web programlama',
 'i̇lk ve acil yardım',
 'i̇şletme',
 'i̇ş sağlığı ve güvenliği',
 'marketing / marketing management, general',
 'business administration',
 'business administration and management, general',
 'çalışma ekonomisi ve endüstri i̇lişkileri',
 'uluslararası i̇lişkiler ve meseleler',
 'i̇şletme ve yönetim, genel',
 'i̇statistik',
 'i̇şletme (lisans)']

In [314]:
it_skills

['elektronik & bilgisayar',
 'bilgisayar donanımı',
 'electrical, electronic and communications engineering technology/technician',
 'math science',
 'computer science and software engineering(csse)',
 'science',
 'bilişim teknolojileri öğretmenliği',
 'bilişim teknolojileri',
 'software engineer',
 'makine resim ve konstrüksiyon',
 'aquaculture engineer',
 'bilişim teknolojileri - veri tabanı programcılığı',
 'mekatronik, robotik ve otomasyon mühendisliği',
 'sofware engineering',
 'software engineering',
 'bilgisayar bilimleri',
 'mathematics & science',
 'data engineering & big data bootcamp',
 'makine mühendisliği',
 'computer engineering',
 'mechatronics engineer',
 'computer science',
 'exchange student, computer science',
 'veri tabanı',
 'bilgisayar muhendisligi',
 'enerji',
 'makine teknolojisi',
 'softwareengineering',
 'electrical electronics engineering',
 'web tasarım , veritabanı sistemleri ve ağ sistemleri',
 'adlî bilişim mühendisliği',
 'yönetim bilişim sistemleri',
 '

In [315]:
# Rename fields_of_study values to it or business, or None
data['fields_of_study'] = data['fields_of_study'].apply(lambda x: 'it' if x in it_skills else 'business' if x in business_skills else "other")
data


,user_id,school_name,degree,fields_of_study,education_count,attended_university
0,0,anadolu üniversitesi,NaN,other,3,1
1,0,adıyaman üniversitesi,NaN,other,3,1
2,0,fırat üniversitesi,NaN,other,3,1
3,1,fırat üniversitesi,yüksek lisans,it,2,1
4,1,fırat üniversitesi,lisans,it,2,1
...,...,...,...,...,...,...
995,587,uniwersytet łódzki,bachelor's degree,it,2,1
996,588,fırat üniversitesi,NaN,other,2,1
997,588,universitatea „petru maior” din târgu-mureș,lisans derecesi,it,2,1
998,589,fırat üniversitesi,lisans derecesi,it,1,1


In [316]:
# Cast degree column into str
data['degree'] = data['degree'].astype(str)

In [317]:
# If degree name contains any of these, then it is a master's degree
master_degree = ['master', 'm.s', 'm.s.', 'm.s.c', 'm.s.c.', 'm.s.c.e', 'm.s.c.e.', 'm.s.e', 'm.s.e.', "yüksek", "doktora", "msc"]

# If degree name contains any of these, then it is a bachelor's degree
bachelor_degree = ['bachelor', 'b.s', 'b.s.', 'b.s.c', 'b.s.c.', 'b.s.c.e', 'b.s.c.e.', 'b.s.e', 'b.s.e.', 'lisans', 'bsc', "associate", "b.e."]

# Apply the above functions to the degree name column
data['degree'] = data['degree'].apply(lambda x: 'master' if any(item in x for item in master_degree) else 'bachelor' if any(item in x for item in bachelor_degree) else "other")

In [318]:
data

,user_id,school_name,degree,fields_of_study,education_count,attended_university
0,0,anadolu üniversitesi,other,other,3,1
1,0,adıyaman üniversitesi,other,other,3,1
2,0,fırat üniversitesi,other,other,3,1
3,1,fırat üniversitesi,master,it,2,1
4,1,fırat üniversitesi,bachelor,it,2,1
...,...,...,...,...,...,...
995,587,uniwersytet łódzki,bachelor,it,2,1
996,588,fırat üniversitesi,other,other,2,1
997,588,universitatea „petru maior” din târgu-mureș,bachelor,it,2,1
998,589,fırat üniversitesi,bachelor,it,1,1


In [319]:
# Add new column number of different universities attended
data['different_university_count'] = data.groupby('user_id')['school_name'].transform('nunique')

# Drop school_name column
data.drop(['school_name'], axis=1, inplace=True)

In [320]:
# If a user has 'master' value in degree column, then set their degree column to master
data['degree'] = data.groupby('user_id')['degree'].transform(lambda x: 'master' if 'master' in x.values else x)

# Do the same for bachelor now, to convert other to bachelor level
data['degree'] = data.groupby('user_id')['degree'].transform(lambda x: 'bachelor' if 'bachelor' in x.values else x)

# Now other is left...

# Rename degree to education_level
data.rename(columns={'degree': 'education_level'}, inplace=True)

In [321]:
data

,user_id,education_level,fields_of_study,education_count,attended_university,different_university_count
0,0,other,other,3,1,3
1,0,other,other,3,1,3
2,0,other,other,3,1,3
3,1,master,it,2,1,1
4,1,master,it,2,1,1
...,...,...,...,...,...,...
995,587,bachelor,it,2,1,2
996,588,bachelor,other,2,1,2
997,588,bachelor,it,2,1,2
998,589,bachelor,it,1,1,1


In [322]:
# Drop the duplicate rows to leave only the first row for each user
data.drop_duplicates(subset='user_id', keep='first', inplace=True)

In [323]:
data

,user_id,education_level,fields_of_study,education_count,attended_university,different_university_count
0,0,other,other,3,1,3
3,1,master,it,2,1,1
5,2,other,it,2,1,2
7,3,bachelor,it,1,1,1
8,4,bachelor,it,1,1,1
...,...,...,...,...,...,...
993,586,bachelor,it,1,1,1
994,587,bachelor,it,2,1,2
996,588,bachelor,other,2,1,2
998,589,bachelor,it,1,1,1


In [324]:
# Save the data
data.to_csv("PreparedData/education.csv", index=False)